In [1]:
#@title Connect to google drive
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#@title Install needed repositories
!pip install mmcv-full==v1.3.9 -f https://download.openmmlab.com/mmcv/dist/cu111/torch1.9.0/index.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.openmmlab.com/mmcv/dist/cu111/torch1.9.0/index.html
     |████████████████████████████████| 34.5 MB 1.2 MB/s 
     |████████████████████████████████| 190 kB 33.1 MB/s 


In [3]:
#@title Clone ViTDope
%cd /content/
! git clone https://github.com/noellelaw/vit-dope
%cd /content/vit-dope
! pip install -r requirements.txt

/content
Cloning into 'vit-dope'...
remote: Enumerating objects: 84, done.
remote: Counting objects: 100% (84/84), done.
remote: Compressing objects: 100% (63/63), done.
remote: Total 84 (delta 23), reused 69 (delta 17), pack-reused 0
Unpacking objects: 100% (84/84), done.
/content/vit-dope
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 50 kB 7.3 MB/s 
     |████████████████████████████████| 314 kB 54.4 MB/s 
  Created wheel for chumpy: filename=chumpy-0.70-py3-none-any.whl size=58286 sha256=4313e75adcfd94be85a94beff0425ab15f39f7f91ec65525622ec3ba3ed07e16
  Stored in directory: /root/.cache/pip/wheels/d9/a2/b8/b8aeeeaeb01b5002085156add1aed832f2fb03e79d0f22dfed
Successfully built chumpy


In [4]:
#@title Install timm and einops
! pip install timm==0.4.9 einops

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 346 kB 36.7 MB/s 
     |████████████████████████████████| 41 kB 627 kB/s 


In [5]:
#@title Imports
import copy
import os
import os.path as osp
from os.path import exists
import matplotlib.pyplot as plt
from scipy.ndimage.filters import gaussian_filter
import time
import warnings
import numpy as np
import json
import datetime
import glob
import cv2
import colorsys
import math

import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
import torchvision.transforms as transforms
from torch.autograd import Variable
import torch.utils.data as data
import torchvision.models as models
from torch.distributions import MultivariateNormal as MVN
from torch.nn.utils import clip_grad_norm_
from torch.nn.functional import cross_entropy

from PIL import Image
from PIL import ImageDraw
from PIL import ImageEnhance

from math import acos
from math import sqrt
from math import pi  

import mmcv
from mmcv import Config, DictAction
from mmcv.utils import get_git_hash
from mmcv.runner import get_dist_info, init_dist, set_random_seed

from collections import OrderedDict
import tempfile
import random
from __future__ import print_function

from models.backbones import ViT
from ndds_dataloader import MultipleVertexJson
from object_detector import ObjectDetector
from core.evaluation.top_down_eval import (keypoint_pck_accuracy,
                            keypoints_from_heatmaps,
                            pose_pck_accuracy)
from models.heads import TopdownHeatmapSimpleHead

In [6]:
#@title Training hyperparameters
DATA_PATH_TEST = '/content/drive/MyDrive/DeepLearning/test_cracker/'
PRETRAINED = '/content/drive/MyDrive/DeepLearning/mae_pretrain_vit_base.pth'
FROM_NET = '/content/drive/MyDrive/DeepLearning/cracker_box_train/net_epoch_39.pth'
YCB_OBJECT = 'cracker_box'
NOISE = 1e-5
BRIGHTNESS = 1e-5
CONTRAST = 1e-5
BATCH_SIZE = 2
IMAGE_SIZE = 256
LEARNING_RATE = 5e-4
EPOCHS = 60
LOG_INTERVAL = 100
SIGMA = 4
OUT_FLDR = '/content/drive/MyDrive/DeepLearning/cracker_box_train'
DATASIZE = None
SAVE = False
NORMAL_IMGS = None
NB_UPDATES = None
NAME_FILE = 'epoch'
MAX_NORM = 1.
NORM_TYPE = 2

In [7]:
#@title Get testing data loaders 
print ("Loading data...")
transform = transforms.Compose([
                          transforms.Resize((IMAGE_SIZE,IMAGE_SIZE)),
                          transforms.ToTensor()])
testingdata = None
if not DATA_PATH_TEST == "":
  testingdata = torch.utils.data.DataLoader(
      MultipleVertexJson(
          root = DATA_PATH_TEST,
          objectofinterest=YCB_OBJECT,
          keep_orientation = True,
          noise = NOISE,
          sigma = SIGMA,
          data_size = DATASIZE,
          save = SAVE,
          transform = transform,
          normal = NORMAL_IMGS,
          target_transform = transforms.Compose([
                                  transforms.Resize(IMAGE_SIZE//4),
              ]),
          ),
      batch_size = BATCH_SIZE, 
      shuffle = True,
      num_workers = 1, 
      pin_memory = True)

Loading data...


In [8]:
#@title Set up ViTDope Network
class ViTDopeNetwork(nn.Module):
  def __init__(
            self,
            pretrained=False,
            numBeliefMap=9,
            numAffinity=16
            ):
    super(ViTDopeNetwork, self).__init__()
    # Set up backbone accordance with ViT-B
    backbone = ViT(img_size=(256,256),
                  patch_size=16,
                  embed_dim=768,
                  depth=12,
                  num_heads=12,
                  ratio=1,
                  use_checkpoint=False,
                  mlp_ratio=4,
                  qkv_bias=True,
                  drop_path_rate=0.3,
    )
    # Init ViT weights from ViT MAE trained on image net
    if not PRETRAINED == '':
        backbone.init_weights(pretrained=PRETRAINED)
    # Set classical decoder head for belief maps
    belief_head = TopdownHeatmapSimpleHead(
        in_channels=768,
        num_deconv_layers=2,
        num_deconv_filters=(256, 256),
        num_deconv_kernels=(4, 4),
        extra=dict(final_conv_kernel=1, ),
        out_channels=numBeliefMap,
        loss_keypoint=dict(type='JointsMSELoss', use_target_weight=True)
    )
    # Set classical decoder head for affity maps
    affinity_head = TopdownHeatmapSimpleHead(
        in_channels=768,
        num_deconv_layers=2,
        num_deconv_filters=(256, 256),
        num_deconv_kernels=(4, 4),
        extra=dict(final_conv_kernel=1, ),
        out_channels=numAffinity,
        loss_keypoint=dict(type='JointsMSELoss', use_target_weight=True)
    )

    self.backbone = nn.Sequential(*[backbone])
    self.belief_head = nn.Sequential(*[belief_head])
    self.affinity_head = nn.Sequential(*[affinity_head])

  # Forward
  def forward(self, x):
    backbone_out = self.backbone(x)
    belief_out = self.belief_head(backbone_out)
    affinity_out = self.affinity_head(backbone_out)
    return belief_out, affinity_out



In [9]:
#@title Set up files for testing & training progress

with open (OUT_FLDR+'/loss_test.csv','w') as file: 
    file.write('epoch,batchid,loss\n')

In [10]:
#@title Set up model and optimizer
net = ViTDopeNetwork()
net = net.to('cuda')
# Load for inference or to resume training
if FROM_NET!= '':
    net.load_state_dict(torch.load(FROM_NET))

In [11]:
#@title Run inference 

def _run_inference(loader,make_belief_debug_img=False,norm_belief=True, overlay_image=True):
    net.eval()

    for batch_idx, targets in enumerate(loader):

        data = Variable(targets['img'].cuda())
        output_belief, output_affinities = net(data) 
        target_belief = Variable(targets['beliefs'].cuda())        
        target_affinity = Variable(targets['affinities'].cuda())

        vertex = output_belief[-1]
        aff = output_affinities[-1]

        # Find objects from network output
        detected_objects = ObjectDetector.find_object_poses(vertex, aff)

        return detected_objects, output_belief[-1], target_belief[-1]


In [12]:
#@title Run inference

if not DATA_PATH_TEST == "":
    data, maps, targets = _run_inference(testingdata,make_belief_debug_img=True)

    for i, map in enumerate(maps):
      out = np.multiply(np.array(map.detach().cpu()), 255)
      filename = f'/content/out_{i}.png'
      cv2.imwrite(filename, out)

    for i, target in enumerate(targets):
      out = np.multiply(np.array(target.detach().cpu()), 255)
      filename = f'/content/target_{i}.png'
      cv2.imwrite(filename, out)

[]
